In [ ]:
import torch
import jax.numpy as jnp
from jax import grad
from jax import jit
import trio

# Byte Pair Encoding Tokenization

Based off of Andrej Kapathy YouTube Video "Let's Build GPT Tokenizer"

``` Place holder for BPE Description```

In [2]:
text = "This is a test string. It has no meaning and serves for testing purposes only. I'll also add the common sentence with every letter next. The quick brown fox jumps over the lazy dog."
encoded_input = text.encode("utf-8")
encoded_input = list(map(int, encoded_input))

In [4]:
def find_pairs(tokens):
    counts = {}

    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [5]:
pairs = find_pairs(encoded_input)

most_freq_pair = max(pairs, key = pairs.get)

def merge_most_frequent(id_list, pair, new_token):
    tokens = []
    i = 0
    while i < len(id_list):
        if i < len(id_list) - 1 and id_list[i] == pair[0] and id_list[i+1] == pair[1]:
            tokens.append(new_token)
            i += 2
        else:
            tokens.append(id_list[i])
            i += 1
    return tokens

# test
print(merge_most_frequent([5, 6, 6, 7, 9, 1], (6, 7), 99))

[5, 6, 99, 9, 1]


In [6]:
vocab_size = 276
num_merges = vocab_size - 256
id_list = list(encoded_input) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = find_pairs(id_list)
  pair = max(stats, key=stats.get)
  new_token = 256 + i
  print(f"merging {pair} into a new token {new_token}")
  ids = merge_most_frequent(id_list, pair, new_token)
  merges[pair] = new_token

merging (115, 32) into a new token 256
merging (115, 32) into a new token 257
merging (115, 32) into a new token 258
merging (115, 32) into a new token 259
merging (115, 32) into a new token 260
merging (115, 32) into a new token 261
merging (115, 32) into a new token 262
merging (115, 32) into a new token 263
merging (115, 32) into a new token 264
merging (115, 32) into a new token 265
merging (115, 32) into a new token 266
merging (115, 32) into a new token 267
merging (115, 32) into a new token 268
merging (115, 32) into a new token 269
merging (115, 32) into a new token 270
merging (115, 32) into a new token 271
merging (115, 32) into a new token 272
merging (115, 32) into a new token 273
merging (115, 32) into a new token 274
merging (115, 32) into a new token 275


In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128]))

In [ ]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode(""))